### Researched Quantities

- Building this model based on a 10-year rainstorm, which is 5.5 inches in 24 hours.
-Here is the link to the sheet with the data about the plants https://1drv.ms/x/c/348b84fad346e9f1/EX-eKXt_eg1HnNybvuDh2aUBIBjV-u4fKmu33ILq_tpZVw?e=BLTTLG 
- Unit conversions:
    - 1 square mile = 640 acres
    - 1 square mile = 27,878,400 square feet
    - 1 foot = 12 inches
    - 1 gallon = 0.13368 cubic feet
- According to HomeAdvisor, it costs between $50 to $250 per linear foot to insall a sewer line. I feel like since it's NYC it'll be more expensive but it's so much that they can probably get pretty good pricing so I'm just going to guess $150 per linear foot.
    https://www.homeadvisor.com/cost/plumbing/install-a-sewer-main/
- according to NYC waterboards, the sewer rate $7.14 / ft^3 = $954479.17 $/ MG in 2024: https://www.nyc.gov/site/nycwaterboard/rates/rates-regulations.page

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

  Activating project at `~/Documents/Programming/Julia/BEE4750/4750project`
   Installed GR_jll ──────────────────── v0.73.9+0
   Installed EpollShim_jll ───────────── v0.0.20230411+1
   Installed HTTP ────────────────────── v1.10.12
   Installed Libmount_jll ────────────── v2.40.2+0
   Installed Unitful ─────────────────── v1.21.1
   Installed MutableArithmetics ──────── v1.6.0
   Installed fzf_jll ─────────────────── v0.56.3+0
   Installed FreeType2_jll ───────────── v2.13.3+1
   Installed JLFzf ───────────────────── v0.1.9
   Installed Xorg_libXau_jll ─────────── v1.0.11+1
   Installed Xorg_libpthread_stubs_jll ─ v0.1.1+1
   Installed Xorg_libxcb_jll ─────────── v1.17.0+1
   Installed Xorg_libXext_jll ────────── v1.3.6+1
   Installed gperf_jll ───────────────── v3.1.1+1
   Installed ExceptionUnwrapping ─────── v0.1.11
   Installed ArgCheck ────────────────── v2.4.0
   Installed Expat_jll ───────────────── v2.6.4+1
   Installed OrderedCollections ──────── v1.7.0
   Installed Libuuid_

In [2]:
#deriving objective function
facilities = ["26th Ward", "Bowery Bay", "Coney Island", "Hunts Point", "Jamaica", "Newtown Creek", "North River", "Oakwood Beach", "Rockaway", "Owls Head", "Wards Island", "Tallman Island", "Port Richmond", "Red Hook"]
design_capacity = [85, 150, 110, 200, 100, 310, 170, 39.9, 45, 120, 275, 80, 60, 60] #MGD
population_served = [283428, 848328, 596326, 684569, 728123, 1068012, 588772, 244918, 90474, 758007, 1061558, 410812, 198128, 192050] #
sanitary_demand = 80*(10^-6)*population_served #MGD = (gal/(day * person)) * people * 10^-6 MGD/(gal/d)


cost_per_mile = 150*5280 #$/ft * ft/mile
distance = [10.25, 9.55, 10, 11, 13, 5.7, 9.7, 11.25, 14.25, 4.45, 8.75, 12.6, 6.15, 3.7] #miles
W_coef = distance*cost_per_mile

storm_height = 5.5/12 #feet
drainage_area = [5907, 15203, 15087, 16664, 25313, 15656, 6030, 10779, 6259, 12947, 12056, 16860, 9665, 3200] #acres = (miles^2)/640
storm_vol = (storm_height * drainage_area)/3.06888 #MGD = ((in/day)*acre)/(3.06888 acre foot / Mgal)
treatment_cost = 954479.17 #dollars / MGD

CSO_cost = 10^10 #dollars (aka 10 billion dollars)


10000000000

In [12]:
#Setting Up Optimization
stormwater_model = Model(HiGHS.Optimizer) #initialize model object

#create variables: amount of wastewater diverted to new plant
@variable(stormwater_model, W[1:14], Bin) #will this plant overflow in a 10-year storm? - boolean
@variable(stormwater_model, Q[1:14] >= 0) #how much water will go to the new plant
@variable(stormwater_model, O[1:14], Bin) #will we get an overflow in a 10-year storm

#create objective function
@objective(stormwater_model, Min, sum(W_coef .* W) + sum(treatment_cost .* Q) + sum(CSO_cost .* O))

#constraints
@constraint(stormwater_model, overflow, O[1] => {storm_vol[1] .+ sanitary_demand[1] <= design_capacity[1]})

overflow : O[1] --> {0 ≤ -819.8771935205025}

In [ ]:
#optimize
optimize!(stormwater_model)
@show value.(W)